# Scan in $(hkl)$ coordinates

This notebook demonstrates how to scan in $(hkl)$ coordinates. It uses the
simulated 4-circle geometry from the `"hkl_soleil"` solver. The wavelength and
sample are whatever the simulator provides as defaults.

**Important**:  It is possible to scan in any combination of reciprocal axes or to
scan in any combination of real-space axes.  You are not allowed to scan in a
mix of reciprocal and real-space axes.

## Setup

First, create the simulated 4-circle diffractometer object (`e4cv`).

In [1]:
from hklpy2 import SimulatedE4CV

e4cv = SimulatedE4CV("", name="e4cv")

Setup Bluesky for running the scans with the `RE` object.  The `bec` object will
show a table of the data collected for each scan.  

For this simple demonstration, we won't add a databroker catalog.

In [2]:
from bluesky import RunEngine, plans as bp
from bluesky.callbacks.best_effort import BestEffortCallback

bec = BestEffortCallback()
RE = RunEngine()
RE.subscribe(bec)
bec.disable_plots()

We'll **import a simulator** (ready to use) from the `ophyd` package as a noisy detector.

In [3]:
from ophyd.sim import noisy_det

## (h10) scan

Scan the (reciprocal space) $h$ axis from -0.5 to +0.5 with $k=1$ and $l=0$.
This is called an $(h10)$ scan.

<details>

The computation to convert reciprocal-space values $(h,k,l)$ into real-space
angles ($\omega$, $\chi$, $\phi$, $2\theta$) is called the `forward()`
transformation.  The transformation is not necessarily unique.  The most common
way to reduce the number of *solutions* is to tell the solver which `mode` to
use.  The `mode` adds an additional pre-designed rule that constrains the
acceptable solutions.  The solver's geometry (in this case `E4CV`) provides the
list of known modes.

Note: Even with a chosen mode, the solution might not be unique.  In such cases,
the first solution returned by the `forward()` transformation is chosen.  The
user can change this by providing a different function for the diffractometer's
`_forward_solution` attribute.  The default is the
`hklpy2.diffract.pick_first_item()` function.

</details>

Here, the diffractometer starts with `"bissector"` mode (requires `tth = 2*omega`).

In [4]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='bissector'


Transient Scan ID: 1     Time: 2024-06-18 10:59:21
Persistent Unique Scan ID: '5f4fc0a9-17e2-4ae3-9fd6-f1e7a4b9b775'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 10:59:21.9 |     -0.500 |      0.977 |
|         2 | 10:59:21.9 |     -0.400 |      1.038 |
|         3 | 10:59:21.9 |     -0.300 |      0.941 |
|         4 | 10:59:21.9 |     -0.200 |      1.077 |
|         5 | 10:59:21.9 |     -0.100 |      1.079 |
|         6 | 10:59:21.9 |      0.000 |      1.061 |
|         7 | 10:59:21.9 |      0.100 |      1.085 |
|         8 | 10:59:21.9 |      0.200 |      0.972 |
|         9 | 10:59:21.9 |      0.300 |      1.028 |
|        10 | 10:59:22.0 |      0.400 |      0.970 |
|        11 | 10:59:22.0 |      0.500 |      1.015 |
+-----------+------------+------------+------------+
generator scan ['5f4fc0a9'

('5f4fc0a9-17e2-4ae3-9fd6-f1e7a4b9b775',)

**Clearly we see** that $h$ has been stepped across the range of -0.5 to +0.5.
Values for the noisy detector have been reported at each step.  But we want to
know about *all* the $hkl$ and angle values so we can observe the effects of
`"bissector"` mode.

### Scan again, showing all $(hkl)$ and real-space axes

Repeat the scan, same as before with a slight variation.  This time, add the
`e4cv` object as an additional detector.

In [5]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det, e4cv], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='bissector'


Transient Scan ID: 2     Time: 2024-06-18 10:59:22
Persistent Unique Scan ID: 'ece958ca-2c83-4391-829d-b542094a508e'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:59:22.2 |     -0.500 |      0.957 |      1.000 |      0.000 |    -33.988 |    -63.435 |     90.000 |    -67.976 |
|         2 | 10:59:22.2 |     -0.400 |      1.031 |      1.000 |      0.000 |    -32.583 |    -68.199 |     90.000 |    -65.165 |
|         3 | 10:59:22.2 |     -0.300 |      1.063 |      1.000 |     -0.000 |    -31.468 |    -73.301 |     90.000 |    -62.935 |
|         4 | 10:59:22.2 |     -0.2

('ece958ca-2c83-4391-829d-b542094a508e',)

## What other modes are available?

In [6]:
e4cv.operator.solver.modes

['bissector',
 'constant_omega',
 'constant_chi',
 'constant_phi',
 'double_diffraction',
 'psi_constant']

## Scan $(h10)$ holding $\omega$ at -30 degrees

Set the mode to `"constant_omega"`, then set $\omega=-30$ degrees.

In [7]:
e4cv.operator.solver.mode = "constant_omega"
e4cv.omega.move(-30)
print(f"{e4cv.omega.position=!r}")

e4cv.omega.position=-30


**Run the scan again** with the same command.

In [8]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det, e4cv], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='constant_omega'


Transient Scan ID: 3     Time: 2024-06-18 10:59:22
Persistent Unique Scan ID: '3f5684c0-a076-4c3a-a496-ccec3d9f7a69'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:59:22.5 |     -0.500 |      0.933 |      1.000 |      0.000 |    -30.000 |    -63.714 |     81.054 |    -67.976 |
|         2 | 10:59:22.5 |     -0.400 |      1.018 |      1.000 |     -0.000 |    -30.000 |    -68.345 |     83.031 |    -65.165 |
|         3 | 10:59:22.5 |     -0.300 |      1.096 |      1.000 |     -0.000 |    -30.000 |    -73.364 |     84.887 |    -62.935 |
|         4 | 10:59:22.5 |    

('3f5684c0-a076-4c3a-a496-ccec3d9f7a69',)

## Scan $(\bar{1}kl)$ holding $\omega$ at -30 degrees

Keep mode as `"constant_omega"` and $\omega=-30$.  Set $h=-1$ and scan $k$ & $l$.

In [9]:
e4cv.h.move(-1)
print(f"{e4cv.operator.solver.mode=!r}")
RE(bp.scan([noisy_det, e4cv], e4cv.k, 0.9, 1.1, e4cv.l, -0.6, -0.4, 11))

e4cv.operator.solver.mode='constant_omega'


Transient Scan ID: 4     Time: 2024-06-18 10:59:22
Persistent Unique Scan ID: '53cb8642-060a-4e88-a115-e4ba3143f595'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_k |     e4cv_l |  noisy_det |     e4cv_h | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:59:22.7 |      0.900 |     -0.600 |      0.993 |     -1.000 |    -30.000 |    -39.821 |     36.793 |    -94.876 |
|         2 | 10:59:22.7 |      0.920 |     -0.580 |      1.086 |     -1.000 |    -30.000 |    -40.796 |     37.124 |    -95.244 |
|         3 | 10:59:22.7 |      0.940 |     -0.560 |      1.032 |     -1.000 |    -30.000 |    -41.770 |     37.424 |    -95.659 |
|         4 | 10:59:22.7 |    

('53cb8642-060a-4e88-a115-e4ba3143f595',)

## Scan $(h10)$ holding $\psi$ at 25 degrees

Set the mode to `"psi_constant"`, then set $\psi=25$ degrees.

TODO: What is $\psi$?  What is $(h_2, k_2, l_2)$?  Is enabled by solver yet?